# Compute projections
Best options is to run `iss project-round` command after each acquisition:

`DATAPATH=BARseq/FIAA48.1b/chambre_03_random`

`iss project-round -p $DATAPATH -n $PREFIX`

where `$DATAPATH` is the path to the data relative to raw data root and `$PREFIX` is the file name prefix and directory name of the round, e.g. `genes_round_1_1`.

This will also start a "plot_overview" job for each round, which will generate a plot of the projected data.
Use this to select a few tiles to use for registration and change `ops.yml` to add these tiles.

## Import requirements

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import iss_preprocess as iss
import numpy as np
import matplotlib.pyplot as plt
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation

## Set path and load options

In [ ]:
data_path = "becalia_rabies_barseq/BRYC65.1d/chamber_13/"
data_path = "BARseq/FIAA48.1b/chambre_03_random"
data_path = "karapir_barseq_5ht/FIAA48.1b/chambre_03_random"
processed_path = iss.io.get_processed_path(data_path)
ops = iss.io.load_ops(data_path)
metadata = iss.io.load_metadata(data_path)

In [ ]:
ops["anchor_binarise_quantile"] is not None

# Illumination correction

Illumination correction is done in two steps: 

- Average each acquisition folder:

This can be done with: `iss create-all-single-averages -p $DATAPATH`

- Average all acquisition averages for each "round" type of data

This can be done with: `iss create-grand-averages -p $DATAPATH`

One can also generate some summary diagnostic plot that will be saved in the `figures`
subfolder of `data_path`

In [ ]:
# Generate diagnostic plot
from iss_preprocess.pipeline import diagnostics

# it takes about 2 minutes to run and save all images
diagnostics.check_illumination_correction(
    data_path, grand_averages=("genes_round",), verbose=False
)

# Normalise channels and rounds

Will try to normalise rolonie brightness across rounds and channels. 

`iss setup-channel-correction -p $DATAPATH`

In [ ]:
# You can redo it here to get a plot:
pixel_dist, norm_factors, norm_factors_raw = iss.pipeline.estimate_channel_correction(
    data_path,
    prefix="barcode_round",
    nrounds=ops["barcode_rounds"],
    fit_norm_factors=ops["fit_channel_correction"],
)
plt.figure()
plt.plot(norm_factors.T)
plt.gca().set_prop_cycle(None)
plt.plot(norm_factors_raw.T, "o")

# Register sequencing rounds

It has 3 steps:

- Register reference tile
- Register all tiles using reference tile initial estimate
- Correct shifts

## Register reference tile

It can be done simply with:

`iss register-ref-tile -p $DATAPATH`

Or from the notebook:

In [ ]:
iss.pipeline.register_reference_tile(
    data_path,
    prefix="genes_round",
    use_slurm=True,
    slurm_folder="/camp/home/znamenp/slurm_logs",
    slurm_options={"mem": "128G", "cpus-per-task": 4, "time": "24:00:00"},
    scripts_name="razi_register_reference_tile",
)

# Check registration

In [ ]:
iss.pipeline.diagnostics.check_ref_tile_registration(
    data_path,
    prefix="genes_round",
)

Start a registration jobs for the reference tile for genes and barcodes:

`iss register-ref-tile -p $DATAPATH -n genes_round`

`iss register-ref-tile -p $DATAPATH -n barcode_round`

where `$DATAPATH` is the path to the data relative to raw data root

Next, estimate shifts for individual tiles:

`iss estimate-shifts -p $DATAPATH -n barcode_round`

`iss estimate-shifts -p $DATAPATH -n genes_round`

Finally, calculate robust regression corrected shifts:

`iss correct-shifts -p $DATAPATH -n barcode_round`

`iss correct-shifts -p $DATAPATH -n genes_round`

# Prepare parameters for OMP and barcode calling

`iss setup-omp -p $DATAPATH`

`iss setup-barcodes -p $DATAPATH`

# Re-compute average spot image

Run OMP on the reference tile and detect spots to generate a new spot sign image.

`iss spot-image -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`

# Check barcode threshold

In [ ]:
stack, bad_pixels = iss.pipeline.load_and_register_tile(
    data_path,
    ops["ref_tile"],
    filter_r=ops["filter_r"],
    prefix="barcode_round",
    suffix=ops["barcode_projection"],
    nrounds=10,
    correct_channels=ops["barcode_correct_channels"],
    corrected_shifts=True,
    correct_illumination=True,
)
spots = iss.segment.detect_spots(np.mean(stack, axis=(2, 3)), threshold=0.1)
plt.figure(figsize=(10, 10))
plt.imshow(np.mean(stack, axis=(2, 3)), cmap="bwr", vmin=-0.1, vmax=0.1)
plt.plot(spots["x"], spots["y"], ".k", markersize=0.5)

# Run OMP
Run `iss extract` to start Slurm jobs to run OMP on each tile:

`iss extract -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`

# Run basecalling

`iss basecall -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`


# Register to reference

To register barcodes to the reference `genes_round_1_1`, we estimate the shift and 
angle at each tile:

`iss register-to-reference -p becalia_rabies_barseq/BRYC65.1d/chamber_13 -n barcode_round`

Then the shifts estimates are corrected using robust regression:

`iss correct-ref-shifts -p becalia_rabies_barseq/BRYC65.1d/chamber_13 -g barcode_round`

And plot diagnostic plot if wanted:

In [ ]:
from iss_preprocess.pipeline.diagnostics import reg_to_ref_estimation

reg_to_ref_estimation(data_path, prefix="barcode_round")


Then merge barcodes across tiles and align to gene rounds:

`iss align-spots -p becalia_rabies_barseq/BRYC65.1d/chamber_13/ -s barcode_round -g barcode_round_1_1 -r genes_round_1_1`


# Merge OMP output

`iss align-spots -p karapir_barseq_5ht/FIAA48.1b/chambre_03_random -s genes_round -g genes_round_1_1 -r genes_round_1_1`

# Plot some barcodes and genes

In [ ]:
import pandas as pd

all_spots = []
genes_spots = []
roi_dims = iss.io.get_roi_dimensions(data_path)
for i, roi in enumerate(roi_dims[:, 0]):
    # save_path = processed_path / f"barcode_round_spots_{roi}.pkl"
    # these_spots = pd.read_pickle(save_path)
    # these_spots["roi"] = roi
    # all_spots.append(these_spots)
    these_spots = pd.read_pickle(processed_path / f"genes_round_spots_{roi}.pkl")
    these_spots["roi"] = roi
    genes_spots.append(these_spots)

# spots = pd.concat(all_spots, ignore_index=True)
genes_spots = pd.concat(genes_spots, ignore_index=True)

In [ ]:
genes = genes_spots["gene"].unique()
rois = genes_spots["roi"].unique()

In [ ]:
xmax = genes_spots["x"].max()
fig = plt.figure(figsize=(10, 120), dpi=300)
fig.patch.set_facecolor("black")
colors = cycle(
    [
        "deepskyblue",
        "darkorange",
        "forestgreen",
        "royalblue",
        "magenta",
        "coral",
        "gold",
        "mediumorchid",
        "red",
        "lawngreen",
    ]
)
for igene, gene in enumerate(genes):
    plt.subplot(len(genes), 1, igene + 1)
    this_gene = genes_spots[
        (genes_spots["gene"] == gene) & (genes_spots["spot_score"] > 0.2)
    ]
    alpha = np.min([2e5 / (this_gene.shape[0] + 1), 1])
    c = next(colors)
    for iroi, roi in enumerate(rois):
        plt.plot(
            this_gene[this_gene["roi"] == roi]["x"] + iroi * xmax * 1.1,
            this_gene[this_gene["roi"] == roi]["y"],
            ",",
            c=c,
            alpha=alpha,
        )
    plt.title(gene, color="white")
    plt.gca().set_aspect("equal", "box")
    plt.gca().invert_yaxis()
    plt.axis("off")
    plt.gca().set_facecolor("black")

(processed_path / "figures" / "genes").mkdir(exist_ok=True, parents=True)
plt.savefig(processed_path / "figures" / "genes" / f"all_genes.png")

In [ ]:
from itertools import cycle
from matplotlib import rcParams

plt.style.use("default")

fig = plt.figure(figsize=(20, 40), dpi=300)
fig.patch.set_facecolor("black")

xmax = genes_spots["x"].max()
ymax = genes_spots["y"].max()
genes = genes_spots["gene"].unique()
prop_cycle = plt.rcParams["axes.prop_cycle"]
rois = genes_spots["roi"].unique()
colors = cycle(
    [
        "deepskyblue",
        "darkorange",
        "forestgreen",
        "royalblue",
        "magenta",
        "coral",
        "gold",
        "mediumorchid",
        "red",
        "lawngreen",
    ]
)

for gene in genes:
    c = next(colors)
    alpha = np.min([1e5 / np.sum(genes_spots["gene"] == gene), 0.2])
    for iroi, roi in enumerate(rois):
        plt.plot(
            genes_spots[(genes_spots["gene"] == gene) & (genes_spots["roi"] == roi)][
                "x"
            ]
            + iroi * 60000,
            genes_spots[(genes_spots["gene"] == gene) & (genes_spots["roi"] == roi)][
                "y"
            ],
            ",",
            c=c,
            alpha=alpha,
        )
plt.gca().set_aspect("equal", "box")
plt.gca().set_facecolor("black")
plt.axis("off")
plt.gca().invert_yaxis()
plt.tight_layout()

In [ ]:
spots = iss.call.correct_barcode_sequences(
    spots[spots["dot_product_score"] > 0.25], max_edit_distance=1
)

from itertools import cycle
from matplotlib import rcParams

plt.style.use("default")

fig = plt.figure(figsize=(20, 40), dpi=300)
fig.patch.set_facecolor("black")

sequence_counts = spots["corrected_bases"].value_counts()
xmax = spots["x"].max()
ymax = spots["y"].max()
genes = genes_spots["gene"].unique()
prop_cycle = plt.rcParams["axes.prop_cycle"]
rois = genes_spots["roi"].unique()
genes_to_plot = [
    "Crym",
    "Ctgf",
    "Igfbp4",
    "Lamp5",
    "Lypd1",
    "Npy",
    "Rgs4",
    "Necab1",
    "Cck",
    "Myl4",
    "Th",
]
for iseq, seq in enumerate(sequence_counts.index[:8]):
    plt.subplot(10, 1, iseq + 1)
    plt.title(seq, color="white")
    plt.gca().set_aspect("equal", "box")
    plt.gca().set_facecolor("black")
    colors = cycle(
        [
            "deepskyblue",
            "darkorange",
            "forestgreen",
            "royalblue",
            "magenta",
            "coral",
            "gold",
            "mediumorchid",
            "red",
            "lawngreen",
        ]
    )

    # for gene in genes_to_plot:
    #     c = next(colors)
    #     alpha = np.min([ 1e5 / np.sum(genes_spots['gene']==gene), 0.2 ])
    #     for iroi, roi in enumerate(rois):
    #         plt.plot(
    #             genes_spots[(genes_spots['gene']==gene) & (genes_spots['roi']==roi)]['x'] + iroi*60000,
    #             genes_spots[(genes_spots['gene']==gene) & (genes_spots['roi']==roi)]['y'],
    #             ',',
    #             c=c,
    #             alpha=alpha,
    #         )
    for iroi, roi in enumerate(rois):
        this_barcode = spots[(spots["corrected_bases"] == seq) & (spots["roi"] == roi)]
        plt.plot(
            this_barcode["x"] + iroi * 60000,
            this_barcode["y"],
            "s",
            markersize=0.2,
            c="white",
        )
    plt.axis("off")
    plt.gca().invert_yaxis()
    plt.tight_layout()

# Generate an overview image and segment cells

Test segmentation parameters:


In [ ]:
roi = 5
(
    stitched_stack_dapi,
    stitched_stack_genes,
    angle,
    shift,
) = iss.pipeline.stitch_and_register(
    data_path, "genes_round_1_1", "barcode_round_10_1", roi=roi, downsample=5
)

In [ ]:
masks = np.load(processed_path / data_path / "masks_1.npy")
im = np.stack(
    [
        stitched_stack_genes[3000:10000, 12000:20000],
        stitched_stack_dapi[3000:10000, 12000:20000],
        masks[3000:10000, 12000:20000] > 0,
    ],
    axis=2,
)
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops["ref_tile"], prefix="genes_round_1_1"
)
spots = iss.pipeline.merge_roi_spots(
    data_path, shift_right, shift_down, tile_shape, iroi=roi, prefix="genes_round"
)
plt.figure(figsize=(50, 50))
plt.imshow(
    iss.vis.to_rgb(
        im,
        colors=[[1, 0, 0], [0, 0, 1], [0, 1, 0]],
        vmax=[400, 200, 1],
        vmin=np.array([30, 0, 0]),
    )
)
plt.plot(spots["x"] - 12000, spots["y"] - 3000, ".r", alpha=1, markersize=0.5)
plt.xlim([0, 8000])
plt.ylim([7000, 0])
plt.axis("off")

In [ ]:
from cellpose.models import CellposeModel
from cellpose.plot import mask_rgb

model = CellposeModel(gpu=True, model_type="cyto", net_avg=False)
masks, flows, styles = model.eval(
    stitched_stack_dapi[11000:12000, 5000:6000],
    rescale=0.55,
    channels=[0, 0],
    flow_threshold=0.4,
    tile=True,
)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(stitched_stack_dapi[11000:12000, 5000:6000], vmax=1000)
plt.subplot(1, 2, 2)
plt.imshow(mask_rgb(masks))

# Run batch segmentation:

`iss segment-all -p $DATAPATH -n $PREFIX`

# Align hybridisation rounds
To run all hybridisation rounds listed in metadata:

`iss estimate-hyb-shifts -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`

`iss correct_hyb_shifts -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`

To run a specific round, e.g. anchor round:

`iss estimate-hyb-shifts -p becalia_rabies_barseq/BRYC65.1d/chamber_13/ -n hybridisation_3_1`

`iss correct_hyb_shifts -p becalia_rabies_barseq/BRYC65.1d/chamber_13/ -n hybridisation_3_1`


In [ ]:
iss.pipeline.setup_hyb_spot_calling(data_path)

# Check hyb spot detection threshold

In [ ]:
stack, bad_pixels = iss.pipeline.load_and_register_hyb_tile(
    data_path,
    tile_coors=ops["ref_tile"],
    prefix="hybridisation_2_1",
    suffix="max",
    correct_illumination=True,
    correct_channels=True,
)

spots = iss.segment.detect_spots(np.max(stack, axis=2), threshold=0.5)
plt.figure(figsize=(20, 20))
plt.imshow(np.max(np.abs(stack), axis=2), vmax=1, vmin=-1, cmap="bwr")
plt.plot(spots["x"], spots["y"], ".", c="k", markersize=3, alpha=0.5)
# plt.xlim([1000,2000])
# plt.ylim([1000,2000])

# Detect hybridisation spots in all tiles

`iss hyb-spots -p becalia_rabies_barseq/BRYC65.1d/chamber_13/`

`iss align-spots -p becalia_rabies_barseq/BRYC65.1d/chamber_13/ -s hybridisation_1_1 -g hybridisation_1_1 -r genes_round_1_1`

`iss align-spots -p becalia_rabies_barseq/BRYC65.1d/chamber_13/ -s hybridisation_2_1 -g hybridisation_2_1 -r genes_round_1_1`
